##Task 1. Create a GKE cluster

In [ ]:
export REPO_NAME=hello-repo
export CLUSTER_NAME=hello-world-ge9u
export ZONE=us-central1-c

In [ ]:
gcloud config set compute/zone $ZONE

In [ ]:
gcloud container clusters create $CLUSTER_NAME \
--release-channel regular \
--cluster-version latest \
--num-nodes 3 \
--min-nodes 2 \
--max-nodes 6 \
--enable-autoscaling --no-enable-ip-alias

In [ ]:
gcloud container clusters update $CLUSTER_NAME --enable-managed-prometheus --zone $ZONE

##Task 2. Enable Managed Prometheus on the GKE cluster

In [ ]:
export NAMESPACE=gmp-dh47
export INTERVAL=50s

In [ ]:
kubectl create ns $NAMESPACE

In [ ]:
gsutil cp gs://spls/gsp510/prometheus-app.yaml .

In [ ]:
cat > prometheus-app.yaml <<EOF

apiVersion: apps/v1
kind: Deployment
metadata:
  name: prometheus-test
  labels:
    app: prometheus-test
spec:
  selector:
    matchLabels:
      app: prometheus-test
  replicas: 3
  template:
    metadata:
      labels:
        app: prometheus-test
    spec:
      nodeSelector:
        kubernetes.io/os: linux
        kubernetes.io/arch: amd64
      containers:
      - image: nilebox/prometheus-example-app:latest
        name: prometheus-test
        ports:
        - name: metrics
          containerPort: 1234
        command:
        - "/main"
        - "--process-metrics"
        - "--go-metrics"
EOF

In [ ]:
kubectl -n $NAMESPACE apply -f prometheus-app.yaml

In [ ]:
gsutil cp gs://spls/gsp510/pod-monitoring.yaml .

In [ ]:
cat > pod-monitoring.yaml <<EOF

apiVersion: monitoring.googleapis.com/v1alpha1
kind: PodMonitoring
metadata:
  name: prometheus-test
  labels:
    app.kubernetes.io/name: prometheus-test
spec:
  selector:
    matchLabels:
      app: prometheus-test
  endpoints:
  - port: metrics
    interval: $INTERVAL
EOF

In [ ]:
kubectl -n $NAMESPACE apply -f pod-monitoring.yaml

##Task 3. Deploy an application onto the GKE cluster

In [ ]:
gsutil cp -r gs://spls/gsp510/hello-app/ .

In [ ]:
cd ~/hello-app

In [ ]:
gcloud container clusters get-credentials $CLUSTER_NAME --zone $ZONE

In [ ]:
kubectl -n $NAMESPACE apply -f manifests/helloweb-deployment.yaml

In [ ]:
cd manifests/

In [ ]:
cat > helloweb-deployment.yaml <<EOF
apiVersion: apps/v1
kind: Deployment
metadata:
  name: helloweb
  labels:
    app: hello
spec:
  selector:
    matchLabels:
      app: hello
      tier: web
  template:
    metadata:
      labels:
        app: hello
        tier: web
    spec:
      containers:
      - name: hello-app
        image: us-docker.pkg.dev/google-samples/containers/gke/hello-app:1.0
        ports:
        - containerPort: 8080
        resources:
          requests:
            cpu: 200m
# [END container_helloapp_deployment]
# [END gke_manifests_helloweb_deployment_deployment_helloweb]
---
EOF

In [ ]:
cd ..

In [ ]:
kubectl delete deployments helloweb  -n $NAMESPACE

In [ ]:
kubectl -n $NAMESPACE apply -f manifests/helloweb-deployment.yaml

In [ ]:
cat > main.go <<EOF
package main

import (
	"fmt"
	"log"
	"net/http"
	"os"
)

func main() {
	// register hello function to handle all requests
	mux := http.NewServeMux()
	mux.HandleFunc("/", hello)

	// use PORT environment variable, or default to 8080
	port := os.Getenv("PORT")
	if port == "" {
		port = "8080"
	}

	// start the web server on port and accept requests
	log.Printf("Server listening on port %s", port)
	log.Fatal(http.ListenAndServe(":"+port, mux))
}

// hello responds to the request with a plain-text "Hello, world" message.
func hello(w http.ResponseWriter, r *http.Request) {
	log.Printf("Serving request: %s", r.URL.Path)
	host, _ := os.Hostname()
	fmt.Fprintf(w, "Hello, world!\n")
	fmt.Fprintf(w, "Version: 2.0.0\n")
	fmt.Fprintf(w, "Hostname: %s\n", host)
}

// [END container_hello_app]
// [END gke_hello_app]

EOF

##Task 4. Create a logs-based metric and alerting policy

In [ ]:
gcloud logging metrics create pod-image-errors \
  --description="awesome lab" \
  --log-filter="resource.type=\"k8s_pod\"
severity=WARNING"

In [ ]:
cat > awesome.json <<EOF_END
{
  "displayName": "Pod Error Alert",
  "userLabels": {},
  "conditions": [
    {
      "displayName": "Kubernetes Pod - logging/user/pod-image-errors",
      "conditionThreshold": {
        "filter": "resource.type = \"k8s_pod\" AND metric.type = \"logging.googleapis.com/user/pod-image-errors\"",
        "aggregations": [
          {
            "alignmentPeriod": "600s",
            "crossSeriesReducer": "REDUCE_SUM",
            "perSeriesAligner": "ALIGN_COUNT"
          }
        ],
        "comparison": "COMPARISON_GT",
        "duration": "0s",
        "trigger": {
          "count": 1
        },
        "thresholdValue": 0
      }
    }
  ],
  "alertStrategy": {
    "autoClose": "604800s"
  },
  "combiner": "OR",
  "enabled": true,
  "notificationChannels": []
}
EOF_END

##Task 5. Update and re-deploy your app

In [ ]:
export SERVICE_NAME=helloweb-service-mrs3

In [ ]:
export PROJECT_ID=$(gcloud config get-value project)
export REGION="${ZONE%-*}"

##Task 6. Containerize your code and deploy it onto the cluster

In [ ]:
cd ~/hello-app/

In [ ]:
gcloud auth configure-docker $REGION-docker.pkg.dev --quiet

In [ ]:
docker build -t $REGION-docker.pkg.dev/$PROJECT_ID/$REPO_NAME/hello-app:v2 .

In [ ]:
docker push $REGION-docker.pkg.dev/$PROJECT_ID/$REPO_NAME/hello-app:v2

In [ ]:
kubectl set image deployment/helloweb -n $NAMESPACE hello-app=$REGION-docker.pkg.dev/$PROJECT_ID/$REPO_NAME/hello-app:v2

In [ ]:
kubectl expose deployment helloweb -n $NAMESPACE --name=$SERVICE_NAME --type=LoadBalancer --port 8080 --target-port 8080

In [ ]:
cd ..

In [ ]:
kubectl -n $NAMESPACE apply -f pod-monitoring.yaml

In [ ]:
gcloud alpha monitoring policies create --policy-from-file="awesome.json"